In [ ]:
import sys
sys.path.append('../')

from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
from src.sample_tiles import *
import pandas as pd

from src.tilenet import make_tilenet
from src.resnet import ResNet18
import torch
from time import time
from torch.autograd import Variable
import random

import rasterio
from rasterio.plot import show

%load_ext autoreload
%autoreload 2
%matplotlib inline

## Extract image

**Place a few images of low pollution season in '../data.nosync/planet-lab-images/low/', same for high.**


In [ ]:
im_dir_path = '../data.nosync/planet-lab-images/'
tile_dir_path = '../data.nosync/planet-lab-tiles/'

im_dir_low = im_dir_path + 'low/'
im_dir_high = im_dir_path + 'high/'
tiles_dir_low = tile_dir_path + 'low/'
tiles_dir_high = tile_dir_path + 'high/'

low_high_dict = {im_dir_low: tile_dir_low,
                 im_dir_high: tile_dir_high}


In [ ]:
for img_dir, tile_dir in low_high_dict.items():
    try:
        os.remove(os.path.join(tile_dir, '.DS_Store'))
    except:
        pass
    try:
        os.remove(os.path.join(img_dir, '.DS_Store'))
    except:
        pass

In [ ]:
# look at a random inage from the im_dir_high
# if it picks .DS_Store, try again or remove that file

img_file = im_dir_high + random.sample(os.listdir(im_dir_high), 1)[0]

with rasterio.open(img_file) as src:
    profile = src.profile

In [ ]:
profile

Make a padding to get coodinates for the tiles

In [ ]:
img_data = load_img(img_file, val_type='uint16', bands_only=False)


img_padded = np.pad(img_data, pad_width=[(tile_radius, tile_radius),
                                             (tile_radius, tile_radius), 
                                             (0,0)],
                        mode='reflect')
img_shape = img_padded.shape

### Visualize a tile form that image

In [ ]:
x, y = sample_anchor(img_shape, tile_radius)
tile = extract_tile(img_padded, x, y, tile_radius)
tile = tile[:-1,:-1] # see size_even


In [ ]:
# need tile_size = 50 to feel into the CNN but can use a bigger one to visualize
tile_size=1000
tile_radius = tile_size // 2

x, y = sample_anchor(img_shape, tile_radius)
tile = extract_tile(img_padded, x, y, tile_radius)
tile = tile[:-1,:-1] # see size_even

In [ ]:
plt.imshow(tile[:,:,[3,2,1]] / 10000.0 )

Here you can play with the settings, still not sure which is the right one

# Sample tiles

Here we are going to sample a couple thousand tiles from low pollution days and a couple thousand from high polution.

The cell below will extract tiles_per_image tiles for each image present in img_dir directory.

In [ ]:
tiles_per_image = 500
tile_size=50
tile_radius = tile_size // 2

for img_dir, tile_dir in low_high_dict.items():

    for image in os.listdir(img_dir):
        print('Saving tiles for ', image, ' in ', tile_dir)
        img = load_img(os.path.join(img_dir, image), val_type='uint16', bands_only=True)
        img_padded = np.pad(img, pad_width=[(tile_radius, tile_radius),
                                            (tile_radius, tile_radius), 
                                            (0,0)],
                            mode='reflect')
        img_shape = img_padded.shape

        for i in range(tiles_per_image):
            x, y = sample_anchor(img_shape, tile_radius)
            tile = extract_tile(img_padded, x, y, tile_radius)
            tile = tile[:-1,:-1] # see size_even
            tile = tile[:,:,[3,2,1,0]] / 10000.0
            np.save(os.path.join(tile_dir, 'planet_tile_' + image[4:15] + '_{}.npy'.format(i)), tile)
            i = i+1


# Load Pre-trained Model

In [ ]:
# Setting up model
in_channels = 4
z_dim = 512
cuda = torch.cuda.is_available()
# tilenet = make_tilenet(in_channels=in_channels, z_dim=z_dim)
# Use old model for now
tilenet = ResNet18()
if cuda: tilenet.cuda()

In [ ]:
# Load parameters
model_fn = '../models/naip_trained.ckpt'
checkpoint = torch.load(model_fn, map_location='cpu')
tilenet.load_state_dict(checkpoint)
tilenet.eval()

Visualize some of the tiles

In [ ]:
tile_dir = tile_dir_low
plt.rcParams['figure.figsize'] = (4,4)
n_tile_plot = 10

for img_dir, tile_dir in low_high_dict.items():
    i = 0
    for filename in random.sample(os.listdir(tile_dir), n_tile_plot):
        tile = np.load(os.path.join(tile_dir, filename))
        print('Tile ' + filename)
        plt.imshow(tile[:,:,[0,1,2]])
        i +=1
        plt.show()

# Remove Black Tiles

Tiles that are sampled in the black area are all black, and have a close to zero mean for all their values. We remove them.

In [ ]:
for img_dir, tile_dir in low_high_dict.items():
    filenames = os.listdir(tile_dir)
    before = len(filenames)
    for filename in filenames:
        tile = np.load(os.path.join(tile_dir, filename))
        if tile.mean() < 0.01:
            os.remove(os.path.join(tile_dir, filename))
    filenames = os.listdir(tile_dir)
    print('Removed ', before - len(os.listdir(tile_dir)), ' black tiles from ', tile_dir)

# Embbed Tiles

In [ ]:
# the features table for low and high PM25 samples

# instanciate
X_low = np.zeros((1, z_dim))
X_high = np.zeros((1, z_dim))
X_dict = {tile_dir_low: X_low,
          tile_dir_high: X_high}

# loop over all tiles and calculate embedding. It takes about 1 min for 500 tiles on my computer, so take it easy
for tile_dir, X in X_dict.items():
    print('Parsing ', tile_dir)
    n_tiles = len(os.listdir(tile_dir))
    X = np.zeros((n_tiles, z_dim))
    # Embed tiles
    t0 = time()
    i = 0
    for filename in os.listdir(tile_dir):
        while i < n_tiles:
            tile = np.load(os.path.join(tile_dir, filename))
            # Get first 4 NAIP channels (5th is CDL mask)
            tile = tile[:,:,:4]
            # Rearrange to PyTorch order
            tile = np.moveaxis(tile, -1, 0)
            tile = np.expand_dims(tile, axis=0)
            # Scale to [0, 1]
            tile = tile / 255
            # Embed tile
            tile = torch.from_numpy(tile).float()
            tile = Variable(tile)
            if cuda: tile = tile.cuda()
            z = tilenet.encode(tile)
            if cuda: z = z.cpu()
            z = z.data.numpy()
            X[i,:] = z
            i += 1
            if i % 100 == 0:
                print(i, ' / ', n_tiles)
        t1 = time()
    X_dict[tile_dir] = X
    print('Embedded {} tiles: {:0.3f}s'.format(n_tiles, t1-t0))
    print('Features table shape : ', X.shape)

# Machine Learning

## Concatenate Features Datatable and get labels


In [ ]:
df_high = pd.DataFrame(X_dict[tile_dir_high])
df_high['label'] = 1

df_low = pd.DataFrame(X_dict[tile_dir_high])
df_low['label'] = 0


features = pd.concat([df_high, df_low], axis = 0)
features.shape

In [ ]:
features = features.reset_index()
labels = features['label']
features = features.drop('label', axis = 1)

## Low VS High Classifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold

This is a grid search that tries to find optimal hypoerparameters on different model. You can also do it more 'by-hand'

In [ ]:
test_size=0.30

random_state = 13
held_out_size = 0.3
n_splits = 3

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
             features, 
             labels, 
             test_size=test_size,
             shuffle = True,
             random_state = random_state)
    
kfold = KFold(n_splits = 3,              
              random_state = random_state)

In [ ]:
binary_models = [LogisticRegression(solver = 'liblinear'),
                 DecisionTreeClassifier(),
                 KNeighborsClassifier(),
                 RandomForestClassifier(random_state = random_state, n_estimators = 33),
                 MLPClassifier(learning_rate='adaptive', learning_rate_init = 0.1, random_state = random_state),
                 MLPClassifier(learning_rate='adaptive', learning_rate_init = 0.1, random_state = random_state)]

parameterScope = [{'penalty' : ['l1', 'l2'], 'C' :  [000.1, 0.005, 0.01, 0.1]},
                  {'min_samples_leaf' : list(range(5, 100, 45))},
                  {'n_neighbors' : list(range(10, 100, 10))},
                  {'max_depth' : list(range(2, 10, 2)), 
                   'n_estimators' : range(20, 85, 30)}]

binary_names = ['Logistic Regression',
                'Decision Tree',
                'K-Nearest Neighbors',
                'Random Forest']

In [ ]:
fitted_binary_models = []
performance = pd.Series()
bestParam = pd.Series()


for model, parameters, name in zip(binary_models, parameterScope, binary_names):
    parameterEstimator = GridSearchCV(model, parameters, cv = kfold, refit = True, return_train_score=False)
    parameterEstimator.fit(X_train, y_train)
    performance[name] = parameterEstimator.best_score_
    bestParam[name] = parameterEstimator.best_params_
    print(name)
    print(performance[name])
    print(bestParam[name])
    fitted_binary_models.append(parameterEstimator.best_estimator_)

bestParam